**NOTE (09/02/24):** Just trying to put together the body of the simulation to make it work. Aiming to use Averill and Law's architecture of what a simulation looks like. 

## Import dependencies

In [2]:
import networkx as nx
import torch
import torch.nn as nn
import random
import uuid
import bisect
import operator

## Constants and containers

In [3]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
PATH = "generator_2.pt"
RANDOM_SEED = 77
ADDRESSES = [i for i in range(1,22)]
BLOCKED = 0 # count of flows blocked
SENT = 0 # count of flows that reached their destination
WAVELENGTHS = 40
MIN_FLOWS_SENT = 1e7

completed_flows = []
blocked_flows = []
events_list = []

## Classes

In [4]:
class Generator(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(2, 16),
            nn.ReLU(),
            nn.Linear(16, 32),
            nn.ReLU(),
            nn.Linear(32, 2),
        )

    def forward(self, x):
        output = self.model(x)
        return output

In [5]:
class Network:
    def __init__(self, graph, num_of_wavelengths):
        self.topology = graph
        self.__allocate_capacity__(num_of_wavelengths)
    
    def __allocate_capacity__(self, num_of_wavelengths):
        self.links = dict()
        for edge in self.topology.edges():
            node1, node2 = edge[0], edge[1] 
            self.links[(node1, node2)] = num_of_wavelengths # updating the weight of each edge, using that to denote the capacity of each link
            # self.links[(node2, node1)] # may be necessary but not sure at this point
          
    def check_capacity(self, node1, node2): 
        # NOTE: Need to add some exception handling because the node pair for the next hop may not correspond with the dictionary of links
        # instead will try it in the correct order, else just switch the arguments around
        try:
            return self.links[(node1, node2)] > 0
        except KeyError:
            return self.links[(node2, node1)] > 0
        
       #     if self.links[(node1, node2)] > 0:
       #         return True
       #     else:
       #         return False
       # except KeyError:
       #     if self.link[(node2, node1)] > 0:
       #         return True
       #     else:
       #         return
    
    def use_capacity(self, node1, node2):
        try: 
            self.links[(node1, node2)] -= 1
        except KeyError:
            self.links[(node2, node1)] -= 1
        # the capacity is going to be used for a finite amount of time
        # If i am assuming that the nodes are evenly spread then I can take the duration of the flow then divide by the amount of hops that it needs to take to get to its destination
        # and that can be the time it takes to move from each node. 
    
    def release_capacity(self, node1, node2):
        try:   
            self.links[(node1, node2)] += 1
        except KeyError: 
            self.links[(node2, node1)] += 1
        
        
    def push_flow(self, flow, node1, node2):
        sufficient_capacity = self.check_capacity(node1, node2)
        if sufficient_capacity:
            self.use_capacity(node1, node2)
            next_event_type = flow.hop()
            return next_event_type, flow 
            # change the flow attribute to mark which node the flow is at currently
        else:
            # print out that Flow ID# has been blocked
            BLOCKED += 1
            
    def find_route(self, src, dst):
        return nx.shortest_path(self.topology, source = src, target = dst)
    

In [34]:
class Flow:
    def __init__(self, network):
        # self.parent_net = network
        self.__create_flow__(network)
        
    def __generate_data__(self):
        random_noise = torch.randn((1, 2), device = DEVICE)
        generated_samples = generator(random_noise)
        generated_samples = generated_samples.cpu().detach().numpy()
        synthetic_dur, synthetic_size = generated_samples[0,0], generated_samples[0,1]
        return synthetic_dur, synthetic_size
        
    def __create_flow__(self, network):
        self.ID = str(uuid.uuid4())
        self.status = "IDLE"
        self.dur, self.size = self.__generate_data__()
        random_addresses = random.sample(ADDRESSES, 2)
        self.src, self.dst = random_addresses[0], random_addresses[1]
        self.current_node = self.src
        self.prev_node = None
        
        # Calculate shortest path
        self.route = network.find_route(self.src, self.dst)
        # Removes the first node in the route which is the starting node for the flow path 
        self.route = self.route[1:] 
        print(self.route)
        print(self.src)
        
        # How long it will take to make each hop, used to schedule the next event time
        self.hop_time = self.dur / len(self.route) 

        
    def hop(self): # Pushes the flow into the network from src to dst/from one hop to the next
        self.status = "SENDING"
        self.prev_node = self.current_node 
        print(f"Sim Clock: {sim_clock} seconds | Flow {self.ID} is moving to node {self.route[0]}.")
        # time.sleep(self.hop_time)
        self.current_node = self.route.pop(0)
        
        if len(self.route) == 0:
            print(f"Flow {self.ID} has reached its destination, node {self.dst}. Now leaving the network...")
            next_event_type = "DEPART"
        else:
            next_event_type = "HOP"
        
        return next_event_type
    
        #if len(self.route) > 1: # it's going to hop to next node
        #    self.prev_node = self.current_node
        #    
        #    print(f"Flow {self.ID} is moving to node {self.route[0]}.")
   # 
    #        # NOTE: should I use time.sleep(self.hop_time) here ???
     #       time.sleep(self.hop_time)
      #      
       #     self.current_node = self.route.pop(0) # removing the next node 
      #      return "hop" # next event to schedule
      #  elif len(self.route) == 1: # it is going to reach its destination with this hop
      #      self.p
      #      return "depart"
      #      
      #  if self.current_node != self.route[0]: # move onto the next node

            
        # need to check the next
        """
        Start a timer that lasts for the flow duration
        
        """
        
    def depart(self): # The flow is now exiting the network
        self.status = "SENT"
        
        
        

In [7]:
class Event:
    def __init__(self, event_time, event_type, flow):
        self.event_type = event_type
        self.event_time = event_time
        # Link a flow to a particular event that occurs, whether it arrives, hops or departs
        self.associated_flow = flow
        # self.event_duration = 0

    

Need to create:
* Initialisation routine
* Timing routine - determines the next events & advances simulation clock
* Library routine - generate the random variates (using the traffic generator)
* Event routine - update system state and statistical counters, & generate future events and add to event list which includes:
    * if its an arrival, calculating the route for the flow, checking the capacity for the next hop, if there is enough capacity then it moves, if not then the flow is blocked = update blocked counter 
    * if its a departure, release the capacity back to the link, if its reached destination then updated completed counter
    * use generator to create a new flow with size and duration and also include the IAT time for the flow to add to the event list
    


In [8]:
def schedule_event(event_time, event_type, associated_flow):
    
    # Create new event
    new_event = Event(event_time, event_type, associated_flow)
    
    # Create key with which the events list will be sorted; will be sorted by event time
    time_key = operator.attrgetter("event_time")
    
    # Inserts a new event into the Events list and maintains its sorted order
    bisect.insort(events_list, new_event, key=time_key) 
    

In [9]:
def initialise_simulation():
    # Set simulation clock to 0
    global sim_clock, generator, network, BLOCKED, SENT
    
    sim_clock = 0
    
    # initialise the generator
    #global generator
    generator = Generator()
    generator.load_state_dict(torch.load(PATH))
    generator = generator.to(DEVICE)
    
    # initialise the network
    G = nx.read_adjlist("UKnet", nodetype=int)
    # global network
    network = Network(G, WAVELENGTHS)
    
    # initialise statistical counters
    #global BLOCKED
    BLOCKED = 0
    #global SENT
    SENT = 0
    
    
    
    # Adds 50 initial events to the events list - they will all be marked as hops 
    for i in range(1):
        
        # Generate new flow
        new_flow = Flow(network)

        # Create and schedule the event
        schedule_event(event_time = i, event_type = "ARRIVAL", associated_flow = new_flow)
    

In [10]:
# Check the next event in the list and advance simulation clock
def timing_routine():
    # Popping the next event from the top of the list
    next_event = events_list.pop(0)
    
    # Accessing the time of the next scheduled event
    advanced_time = next_event.event_time
    
    # Advancing the simulation clock to the time of the next scheduled event
    global sim_clock
    sim_clock = advanced_time
    
    return next_event


In [49]:
def event_routine(event):
    global network
    global sim_clock
    global events_list
    global SENT
    current_flow = event.associated_flow
    if event.event_type == "ARRIVAL" or event.event_type == "HOP":
        
        # Accessing the current node and next node in the flow path
        src, dst = current_flow.current_node, current_flow.route[0]
        
        if event.event_type == "HOP": # It has hopped before, and needs to release the capacity from the previous link
            network.release_capacity(current_flow.prev_node, current_flow.current_node)
        
        # Allow the network to push the flow to the next node
        new_event_type, updated_flow = network.push_flow(current_flow, src, dst)
        
        schedule_event(sim_clock+current_flow.hop_time, new_event_type, associated_flow = updated_flow)
            
    else: # It's a departure event
        # Release the capacity from the last hop
        network.release_capacity(current_flow.prev_node, current_flow.current_node)
        SENT += 1
        

new_flow = Flow()
print(f"Flow size: {new_flow.size} bytes\nFlow Duration: {new_flow.dur} milliseconds\nSource address: {new_flow.src}\nDestination address: {new_flow.dst}\nCurrent status: {new_flow.status}")
new_flow.push()
print(new_flow.status)


In [13]:
def main():
    
    # pass
    initialise_simulation()
    # invoke initialisation routine
        # sim clock = 0
        # initialise the network
        # initialise the statistical counters
        # initialise the event list
    
    
    # Run simulation until a certain amount of flows have been sent across the network
    while (SENT + BLOCKED) < MIN_FLOWS_SENT:
        
        next_event = timing_routine()
        
    
        # invoke timing routine - checking what the next event is and advancing simulation clock to that time
            # check the events list (I will be making the events list automatically sorted so that the next event time is at the front)
            # might make an Event class to store what exactly is going to happen at that point in time that stores the flow:
                # will it be an initial arrival, an intermediary hop, or a departure
                # if arrival - just make new HOP event to move to the next node
                # if hop - new event to move to the next node
                # if departure - then no new events added, just update statistical counters
    
        # invoke event routine - update system state, statistical counters, generate future events, add future events to events list
            # will invoke a specific function, either: arrival(), hop() or departure()
            # checking the link capacity
                # yes capacity - hops to next node
                # no capacity - blocked
            # add a new event of type arrival, hop or departure
        
    # compute and write report on estimates of interest
        # how many flows were sent in total
        # how many flows were blocked
        # how many flows reached their destination
    


In [42]:
# Main function for testing purposes
def main2():
    print("hello")
    initialise_simulation()
    while events_list:
        next_event = timing_routine()
        event_routine(next_event)
    print(f"Total flows travelled through the network: {BLOCKED + SENT}")
    print(f"Total flows successfully sent to its destination: {SENT}")
    print(f"Total flows blocked: {BLOCKED}")

In [50]:
if __name__ == "__main__":
    main2()

hello
[20, 3]
9
Sim Clock: 0 seconds | Flow 7056c445-590d-4d77-89fe-5ac8f9f27117 is moving to node 20.
Sim Clock: 2062.2294921875 seconds | Flow 7056c445-590d-4d77-89fe-5ac8f9f27117 is moving to node 3.
Flow 7056c445-590d-4d77-89fe-5ac8f9f27117 has reached its destination, node 3. Now leaving the network...
Total flows travelled through the network: 1
Total flows successfully sent to its destination: 1
Total flows blocked: 0


In [36]:
for event in events_list:
    print(event.associated_flow.ID)

In [41]:
print(network.links)

{(1, 3): 40, (1, 2): 40, (1, 20): 40, (1, 4): 40, (1, 6): 40, (1, 8): 40, (1, 14): 40, (3, 2): 40, (3, 20): 40, (3, 19): 40, (2, 19): 40, (20, 9): 40, (20, 21): 40, (4, 19): 40, (4, 7): 40, (4, 6): 40, (4, 5): 40, (6, 5): 40, (6, 8): 40, (8, 7): 40, (8, 9): 40, (8, 10): 40, (14, 10): 40, (14, 15): 40, (7, 9): 39, (9, 21): 40, (9, 16): 40, (9, 15): 40, (9, 11): 40, (10, 11): 40, (10, 13): 40, (21, 16): 40, (16, 18): 40, (16, 17): 40, (15, 13): 40, (15, 17): 40, (11, 12): 40, (13, 12): 40, (17, 18): 40}
